In [ ]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras import layers,Model
from tensorflow.keras.optimizers import RMSprop,SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.applications import Xception,ResNet50V2,VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import random
import os 
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
import imagehash


In [ ]:
train_path='../input/petfinder-pawpularity-score/train'
test_path='../input/petfinder-pawpularity-score/test'
train=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
#reproducible results
def seed(s):
    random.seed(s)
    tf.random.set_seed(s)
    np.random.seed(s)
    os.environ['PYTHONHASHSEED']=str(s)
globalseed=43
seed(globalseed)

In [ ]:
def path(Id):
    return f'../input/petfinder-pawpularity-score/train/{Id}.jpg'
train['path']=train['Id'].apply(path)

In [ ]:
train.head()

In [ ]:
def hash_value(path):
    img=Image.open(path)
    image=imagehash.phash(img)
    return image.hash.reshape(-1).astype(np.int64)
tf_train['hash']=tf_train['path'].apply(hash_value)

In [ ]:
def duplicate(threshold=0.90):
    duplicate_set=set()
    for id,phash in enumerate(tf_train['hash']):
        for id_other,phash_other in enumerate(tf_train['hash']):
            similarity=(phash==phash_other).mean()
            if id!=id_other and similarity>threshold and not (duplicate_set.intersection([id,id_other])):
                duplicate_set.update([id,id_other])
    return duplicate_set
duplicate_img=duplicate()

In [ ]:
print(f'duplicate images : {len(duplicate_img)}')
tf_train=tf_train.drop(duplicate_img).reset_index(drop=True)

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,width_shift_range=0.25,height_shift_range=0.2,rotation_range=25,horizontal_flip=True,fill_mode='nearest',validation_split=0.2)
train_data=train_datagen.flow_from_dataframe(tf_train,x_col='path',y_col='Pawpularity',target_size=(256,256),color_mode='rgb',class_mode='raw',batch_size=128,seed=globalseed,subset='training')
validation=train_datagen.flow_from_dataframe(tf_train,x_col='path',y_col='Pawpularity',taget_size=(256,256),color_mode='rgb',class_mode='raw',batch_size=128,seed=globalseed,subset='validation')


In [ ]:
from tensorflow import keras
model1=ResNet50V2(input_shape=(256,256,3),include_top=False,weights=None)
weights='../input/tf-keras-pretrained-model-weights/No Top/resnet50v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
model1.load_weights(weights)
for layers in model1.layers:
    layers.trainable=False
opt1=model1.output
x=keras.layers.Flatten()(opt1)
x=keras.layers.Dense(512,activation='relu')(x)
x=keras.layers.Dropout(0.2)(x)
x=keras.layers.Dense(512,activation='relu')(x)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Dropout(0.2)(x)
x=keras.layers.Dense(1,name='pawpularity')(x)
resnet_model=Model(model1.input,x)

tf.keras.utils.plot_model(resnet_model)
resnet_model.summary()

In [ ]:
model2=Xception(input_shape=(256,256,3),include_top=False,weights=None)
weights1='../input/tf-keras-pretrained-model-weights/No Top/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
model2.load_weights(weights1)
for layers in model2.layers:
    layers.trainable=False
opt2=model2.output
x=keras.layers.Flatten()(opt2)
x=keras.layers.Dense(512,activation='relu')(x)
x=keras.layers.Dropout(0.2)(x)
x=keras.layers.Dense(512,activation='relu')(x)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Dropout(0.2)(x)
x=keras.layers.Dense(1,name='pawpularity')(x)
xception_model=Model(model2.input,x)


tf.keras.utils.plot_model(xception_model)
xception_model.summary()

In [ ]:
model3=VGG19(input_shape=(256,256,3),include_top=False,weights=None)
weights2='../input/tf-keras-pretrained-model-weights/No Top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
model3.load_weights(weights2)
for layers in model3.layers:
    layers.trainable=False
opt3=model3.output
x=keras.layers.Flatten()(opt3)
x=keras.layers.Dense(512,activation='relu')(x)
x=keras.layers.Dropout(0.2)(x)
x=keras.layers.Dense(512,activation='relu')(x)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Dropout(0.2)(x)
x=keras.layers.Dense(1,name='pawpularity')(x)
vgg_model=Model(model3.input,x)

tf.keras.utils.plot_model(vgg_model)
vgg_model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
lr_rate=ExponentialDecay(initial_learning_rate=0.001,decay_steps=1000,decay_rate=0.95,staircase=True)
callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)


In [ ]:
resnet_model.compile(optimizer=SGD(lr_rate),loss=tf.keras.losses.MeanSquaredError(),metrics=[tf.keras.metrics.RootMeanSquaredError()])
xception_model.compile(optimizer=SGD(lr_rate),loss=tf.keras.losses.MeanSquaredError(),metrics=[tf.keras.metrics.RootMeanSquaredError()])
vgg_model.compile(optimizer=SGD(lr_rate),loss=tf.keras.losses.MeanSquaredError(),metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
hist1=resnet_model.fit(train_data,epochs=12,validation_data=validation,callbacks=[callback])

In [ ]:
hist2=xception_model.fit(train_data,epochs=12,validation_data=validation,callbacks=[callback])

In [ ]:

hist3=vgg_model.fit(train_data,epochs=12,validation_data=validation,callbacks=[callback])

In [ ]:
def plot(hist):
    loss = hist2.history['loss']
    val_loss = hist2.history['val_loss']
    rmse = hist2.history['root_mean_squared_error']
    val_rmse = hist2.history['val_root_mean_squared_error']
    
    x_axis = range(1, len(loss) + 1)
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True)
    
    ax1.plot(x_axis, loss, label='Training')
    ax1.plot(x_axis, val_loss, label='Validation')
    ax1.set_title('MSE Loss')
    ax1.legend()
    ax2.plot(x_axis, rmse, label='Training')
    ax2.plot(x_axis, val_rmse, label='Validation')
    ax2.set_title('Root Mean Squared Error')
    ax2.set_xlabel('Epochs')
    ax2.legend()
    plt.show()
    
print('his1 :')
plot(hist1)

print('/nhist2 :')
plot(hist2)

print('/nhist3 :')
plot(hist3)

In [ ]:
#Test Data
test_datagen=ImageDataGenerator(rescale=1./255)
test_data=test_datagen.flow_from_directory('../input/petfinder-pawpularity-score',classes=['test'],target_size=(256,256))

In [ ]:
filenames=[]
for i in test_data.filenames:
    filenames.append(i.split('/')[1].split('.')[0])
filenames

In [ ]:
# Ensemble Prediction
y_predict1=resnet_model.predict(test_data)
y_predict2=xception_model.predict(test_data)
y_predict3=vgg_model.predict(test_data)

predict=np.mean([y_predict1,y_predict2,y_predict3],axis=0)

In [ ]:
submission=pd.DataFrame()
submission['Id']=filenames
submission['Pawpularity']=pd.Series(predict.reshape(-1))
submission